In [2]:
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras

import numpy as np
import os
import time
import random

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#NO EXECUTAR

#path_to_fileDL ='/content/drive/My Drive/Treball de recerca/Datasets/wikisent2.txt'
#lines = open(path_to_fileDL).readlines()
#random.shuffle(lines)
#del lines[len(lines)//20:]
#open(path_to_fileDL, 'w').writelines(lines)

In [5]:
path_to_fileDL ='/content/drive/My Drive/Treball de recerca/Datasets/wikisent2.txt'
text = open(path_to_fileDL, 'rb').read().decode(encoding='utf-8')
print('Longitud del text:        {} caràters'.format(len(text)))
vocab = sorted(set(text))

print ('El text està compost d’aquests {} caràcters:'.format(len(vocab)))
print (vocab)

Longitud del text:        46759710 caràters
El text està compost d’aquests 96 caràcters:
['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~']


In [6]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

for char,_ in zip(char2idx, range(len(vocab))):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))

  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '"' :   3,
  '#' :   4,
  '$' :   5,
  '%' :   6,
  '&' :   7,
  "'" :   8,
  '(' :   9,
  ')' :  10,
  '*' :  11,
  '+' :  12,
  ',' :  13,
  '-' :  14,
  '.' :  15,
  '/' :  16,
  '0' :  17,
  '1' :  18,
  '2' :  19,
  '3' :  20,
  '4' :  21,
  '5' :  22,
  '6' :  23,
  '7' :  24,
  '8' :  25,
  '9' :  26,
  ':' :  27,
  ';' :  28,
  '<' :  29,
  '=' :  30,
  '>' :  31,
  '?' :  32,
  '@' :  33,
  'A' :  34,
  'B' :  35,
  'C' :  36,
  'D' :  37,
  'E' :  38,
  'F' :  39,
  'G' :  40,
  'H' :  41,
  'I' :  42,
  'J' :  43,
  'K' :  44,
  'L' :  45,
  'M' :  46,
  'N' :  47,
  'O' :  48,
  'P' :  49,
  'Q' :  50,
  'R' :  51,
  'S' :  52,
  'T' :  53,
  'U' :  54,
  'V' :  55,
  'W' :  56,
  'X' :  57,
  'Y' :  58,
  'Z' :  59,
  '[' :  60,
  '\\':  61,
  ']' :  62,
  '^' :  63,
  '_' :  64,
  '`' :  65,
  'a' :  66,
  'b' :  67,
  'c' :  68,
  'd' :  69,
  'e' :  70,
  'f' :  71,
  'g' :  72,
  'h' :  73,
  'i' :  74,
  'j' :  75,
  'k' :  76,

In [7]:
text_as_int = np.array([char2idx[c] for c in text])

In [8]:
print ('texto: {}'.format(repr(text[:50])))
print ('{}'.format(repr(text_as_int[:50])))

texto: 'In 2014, Montenegro was chosen to serve as Majorit'
array([42, 79,  1, 19, 17, 18, 21, 13,  1, 46, 80, 79, 85, 70, 79, 70, 72,
       83, 80,  1, 88, 66, 84,  1, 68, 73, 80, 84, 70, 79,  1, 85, 80,  1,
       84, 70, 83, 87, 70,  1, 66, 84,  1, 46, 66, 75, 80, 83, 74, 85])


In [9]:
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

seq_length = 100
 
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [10]:
for item in sequences.take(10):
  print(repr(''.join(idx2char[item.numpy()])))

'In 2014, Montenegro was chosen to serve as Majority Leader in the 2015-2017 session.\nThe Yale Alumni '
'Magazine reported in November 2008 that the School admitted sixty-five applicants from among 1,142 fo'
'r its class of 2010, and that fifty-six enrolled.\nJohn Cale (born 9 March 1942) is a Welsh musician, '
'composer, singer-songwriter and record producer who was a founding member of the experimental rock ba'
'nd The Velvet Underground in the 1960s.\nRestless Trio is a 1982 Hong Kong action comedy television se'
'ries produced by TVB and starring Andy Lau, Stephen Tung and Patricia Chong.\nIn computer programming,'
' negative cache is a cache that also stores "negative" responses, i.e. failures.\nThis version of the '
'character was created by Steve Ditko and first appeared as a back-up feature in Captain Atom #83 (Nov'
"ember 1966), with Gary Friedrich scripting from Ditko's conception and plot.\nThe various trusts work "
'together and with local authorities and voluntary organiz

In [11]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)


In [12]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'In 2014, Montenegro was chosen to serve as Majority Leader in the 2015-2017 session.\nThe Yale Alumni'
Target data: 'n 2014, Montenegro was chosen to serve as Majority Leader in the 2015-2017 session.\nThe Yale Alumni '


In [13]:

BATCH_SIZE = 64

BUFFER_SIZE = len(text)

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

print (dataset)

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>


In [14]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

vocab_size = len(vocab)
embedding_dim = 256
LSTM_units = 1024

model = Sequential()
model.add(Embedding(input_dim=vocab_size, 
                    output_dim=embedding_dim, 
                    batch_input_shape=[BATCH_SIZE, None]))
model.add(LSTM(LSTM_units,
                return_sequences=True,
                stateful=True,))
model.add(Dense(vocab_size))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           24576     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 96)            98400     
Total params: 5,369,952
Trainable params: 5,369,952
Non-trainable params: 0
_________________________________________________________________


In [15]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss)

EPOCHS=5
history = model.fit(dataset, epochs=EPOCHS,verbose= 1)
weights = model.get_weights()

Epoch 1/5
7233/7233 [==============================] - 392s 54ms/step - loss: 1.4868
Epoch 2/5
7233/7233 [==============================] - 393s 54ms/step - loss: 1.2466
Epoch 3/5
7233/7233 [==============================] - 391s 54ms/step - loss: 1.2049
Epoch 4/5
7233/7233 [==============================] - 390s 54ms/step - loss: 1.1843
Epoch 5/5
7233/7233 [==============================] - 390s 54ms/step - loss: 1.1719


In [16]:
model_prediction = Sequential()
model_prediction.add(Embedding(input_dim=vocab_size, 
                    output_dim=embedding_dim, 
                    batch_input_shape=[1, None]))
model_prediction.add(LSTM(LSTM_units,
                return_sequences=True,
                stateful=True,))
model_prediction.add(Dense(vocab_size))
model_prediction.set_weights(weights)

In [29]:
def generate_text(model, start_string):

  num_generate = 300
  input_eval = [char2idx[s] for s in start_string]

  input_eval = tf.expand_dims(input_eval, 0)
  text_generated = []


  temperature = 0.5

  model.reset_states()
  for i in range(num_generate):
      predictions = model_prediction(input_eval)
      
      predictions = tf.squeeze(predictions, 0)

      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()


      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [33]:
print(generate_text(model, start_string=u"Amazon"))

Amazon Company is a private hotel in the Cape County Courthouse in the U.S. state of West Virginia.
It is also the first for an American main character of the state association of the United States Army Air Force (NRL).
The song was produced by Stephen Kershall and was released by the University of Dance 
